In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.utils import shuffle
import sklearn 
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings('ignore')




### Wheeze_188 Distribution

In [9]:
Total = np.unique(pd.read_csv("./188_wheeze.txt", header=None)[0].values)
name = []
for i in Total:
    name.append(i[:4])

unique = []
before_after = []
for i in Total:
    if np.sum((np.array(name, dtype = int) == int(i[:4]))) == 1:
        unique.append(i)
    else:
        before_after.append(i) 

Asthmatic_Female = []
Asthmatic_Male = []
Healthy_Male = []
Healthy_Female = []
for file in unique:
    if file.find('sthma') !=-1:
        if file.find("_M_")!=-1:
            Asthmatic_Male.append(file)
        if file.find("_F_")!=-1:
            Asthmatic_Female.append(file)
    if file.find("_C_")!=-1:
        if file.find("_M_")!=-1:
            Healthy_Male.append(file)
        if file.find("_F_")!=-1:
            Healthy_Female.append(file)

In [88]:
print(f"Total = {len(Total)}")
print(f"Asthmatic_male = {len(Asthmatic_Male)}")
print(f"Asthmatic_female = {len(Asthmatic_Female)}")
print(f"Healthy_Male = {len(Healthy_Male)}")
print(f"Healthy_female = {len(Healthy_Female)}")
print(f"Before_after = {len(before_after)}")

Total = 188
Asthmatic_male = 57
Asthmatic_female = 45
Healthy_Male = 42
Healthy_female = 38
Before_after = 6


### Breathe

#### Helper functions

In [15]:
def train_test_set(Asthmatic_Female, Asthmatic_Male, Healthy_Female, Healthy_Male, before_after):
    random.seed(0)
    test_set = np.concatenate(((np.random.choice(np.array(Asthmatic_Male), replace=False, size=(7)),
                                np.random.choice(np.array(Asthmatic_Female), replace=False, size=(7)),
                                np.random.choice(np.array(Healthy_Male), replace=False, size=(7)),
                                np.random.choice(np.array(Healthy_Female), replace=False, size=(7)))))
    train_set = np.concatenate((np.array(Asthmatic_Male),
                                np.array(Asthmatic_Female),
                                np.array(Healthy_Male),
                                np.array(Healthy_Female)))
    train_set = np.setdiff1d(train_set,test_set)
#     np.array(before_after).shape
    train_set = np.concatenate((train_set, np.array(before_after)))
    return train_set, test_set

In [14]:
# train_test Csv
def train_test_csv(sound_dataframe):
    Total = np.unique(sound_dataframe.iloc[:,sound_dataframe.shape[1]-1])
    name = []
    for i in Total:
        name.append(i[:4])

    unique = []
    before_after = []
    for i in Total:
        if np.sum((np.array(name, dtype = int) == int(i[:4]))) == 1:
            unique.append(i)
        else:
            before_after.append(i) 

    Asthmatic_Female = []
    Asthmatic_Male = []
    Healthy_Male = []
    Healthy_Female = []
    for file in unique:
        if file.find('sthma') !=-1:
            if file.find("_M_")!=-1:
                Asthmatic_Male.append(file)
            if file.find("_F_")!=-1:
                Asthmatic_Female.append(file)
        if file.find("_C_")!=-1:
            if file.find("_M_")!=-1:
                Healthy_Male.append(file)
            if file.find("_F_")!=-1:
                Healthy_Female.append(file)

                

    Train, Test = train_test_set(Asthmatic_Female, Asthmatic_Male, Healthy_Female, Healthy_Male, before_after)     

    test_dataframe = pd.DataFrame()
    for i, line in enumerate(Test):
        A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Test[i]))]
        test_dataframe = pd.DataFrame.append(test_dataframe,A)
        
        
    train_dataframe = pd.DataFrame()
    for i, line in enumerate(Train):
        A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Train[i]))]
        train_dataframe = pd.DataFrame.append(train_dataframe,A) 
        
        
    
    return train_dataframe, test_dataframe

In [16]:
#Machine Learning Algorithm (MLA) Selection and Initialization
def MLA_selection(sound_dataframe, sound, path):    
    MLA = [
        #Ensemble Methods
#         ensemble.AdaBoostClassifier(),
#         ensemble.BaggingClassifier(),
#         ensemble.ExtraTreesClassifier(),
        ensemble.GradientBoostingClassifier(),
        ensemble.RandomForestClassifier(),

        #Gaussian Processes
#         gaussian_process.GaussianProcessClassifier(),

        #GLM
#         linear_model.LogisticRegressionCV(),
#         linear_model.PassiveAggressiveClassifier(),
#         linear_model.RidgeClassifierCV(),
#         linear_model.SGDClassifier(),
#         linear_model.Perceptron(),

        #Navies Bayes
#         naive_bayes.BernoulliNB(),
#         naive_bayes.GaussianNB(),

        #Nearest Neighbor
#         neighbors.KNeighborsClassifier(),

        #SVM
        svm.SVC(probability=True),
#         svm.NuSVC(probability=True),
        svm.LinearSVC(),

        #Trees    
#         tree.DecisionTreeClassifier(),
# #         tree.ExtraTreeClassifier(),

        #Discriminant Analysis
#         discriminant_analysis.LinearDiscriminantAnalysis(),
#         discriminant_analysis.QuadraticDiscriminantAnalysis(),


        XGBClassifier(max_depth = 8,
                     subsample = 0.8,
                     learning_rate = 0.01,
                     n_estimators = 450,
                     min_child_weight = 1)
#         XGBClassifier()
        ]


    MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy','MLA Train Accuracy Mean', 'MLA Test Accuracy', 'MLA Test Accuracy Mean','MLA Test Accuracy Std' ]
    MLA_compare = pd.DataFrame(columns = MLA_columns)
    row_index = 0
    for alg in MLA:
        MLA_name = alg.__class__.__name__
        MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
        MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
        test = []
        train = []
        for i in tqdm(range(0,10)):
            train_csv, test_csv = pd.read_csv(path+'/train'+sound+str(i)+'.csv'), pd.read_csv(path+'./test'+sound+str(i)+'.csv')
            X_train = train_csv.iloc[:,:sound_dataframe.shape[1]-2]
            y_train = train_csv.iloc[:,sound_dataframe.shape[1]-2]
            X_test = test_csv.iloc[:,:sound_dataframe.shape[1]-2]
            y_test = test_csv.iloc[:,sound_dataframe.shape[1]-2]
            #Scaling
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            X_train = pd.DataFrame(X_train)
            X_test = pd.DataFrame(X_test)

            alg.fit(X_train,y_train)
            test.append(metrics.accuracy_score(y_test,alg.predict(X_test)))

            train.append(metrics.accuracy_score(y_train,alg.predict(X_train)))
            
#             if alg.__class__.__name__ == 'XGBClassifier':
#                 filename = './CMN/XGB_model'+ str(i) + '.model'
#                 pickle.dump(alg, open(filename, 'wb'))
#                 train_csv.to_csv('./CMN/Train_XGB'+str(i)+'.csv', index=False)
                
        
        MLA_compare.loc[row_index, 'MLA Train Accuracy'] = train
        MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = np.mean(train)
        MLA_compare.loc[row_index, 'MLA Test Accuracy'] = test
        MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = np.mean(test)
        MLA_compare.loc[row_index, 'MLA Test Accuracy Std'] = np.std(test)


        row_index+=1

    
    MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
    
    return MLA_compare

In [78]:
# # Making Sets
# def make_and_save_sets(sound_dataframe, sound):
#     for i in range(0,10):
#         train_csv, test_csv = train_test_csv(sound_dataframe)
#         train_csv.to_csv('./breathe_set_csv/train'+sound+str(i)+'.csv', index=False)
#         test_csv.to_csv('./breathe_set_csv/test'+sound+str(i)+'.csv', index=False)

def make_legit_sets(sound_dataframe, sound, path):
    for i in range(0,10):
        Train = np.loadtxt(f"./{path}/Train/Train_{i}.txt", dtype= 'str')
        Test = np.loadtxt(f"./{path}/Test/Test_{i}.txt", dtype= 'str')


        test_dataframe = pd.DataFrame()
        for j, line in enumerate(Test):
            A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Test[j]))]
            test_dataframe = pd.DataFrame.append(test_dataframe,A)

        train_dataframe = pd.DataFrame()
        for j, line in enumerate(Train):
            A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Train[j]))]
            train_dataframe = pd.DataFrame.append(train_dataframe,A)

        train_dataframe.to_csv('./breathe_set_csv/train'+sound+str(i)+'.csv', index=False)
        test_dataframe.to_csv('./breathe_set_csv/test'+sound+str(i)+'.csv', index=False)
    return None

In [7]:
Wheeze_dataframe = pd.read_csv("./MFCCs_stasts_Csv/Wheeze.csv")
Breath_dataframe = pd.read_csv("./breath_csv/Breath.csv")

In [33]:
# make_and_save_sets(Wheeze_dataframe, 'Wheeze')

In [36]:
MLA_Wheeze = MLA_selection(Wheeze_dataframe, 'wheeze', "./Set_CSV/")

  0%|          | 0/10 [00:00<?, ?it/s]

[15:30:56] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:06<00:58,  6.50s/it]

[15:31:00] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:10<00:41,  5.13s/it]

...

 80%|████████  | 8/10 [00:33<00:07,  3.83s/it]

[15:31:27] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:36<00:03,  3.68s/it]

[15:31:30] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:40<00:00,  4.00s/it]


In [37]:
MLA_Wheeze

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9920863309352518, 0.9934306569343065, 0.991...",0.991533,"[0.8968609865470852, 0.8641975308641975, 0.751...",0.860558,0.0612982
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9992805755395684, 1.0, 0.9992690058479532, ...",0.999562,"[0.8923766816143498, 0.8683127572016461, 0.767...",0.859775,0.0552414
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8834080717488789, 0.8600823045267489, 0.763...",0.858964,0.0582834
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9784172661870504, 0.9773722627737226, 0.978...",0.97459,"[0.8609865470852018, 0.831275720164609, 0.7632...",0.82091,0.040913
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8985611510791367, 0.9138686131386862, 0.921...",0.902892,"[0.8295964125560538, 0.7366255144032922, 0.706...",0.794729,0.0560749


In [37]:
# make_and_save_sets(Breath_dataframe, 'Breath')

In [38]:
# MLA_Breath = MLA_selection(Breath_dataframe, 'Breath', "./Set_CSV/")

  0%|          | 0/10 [00:00<?, ?it/s]

[10:34:58] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:00<00:08,  1.12it/s]

[10:34:59] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:01<00:07,  1.14it/s]

...

 80%|████████  | 8/10 [00:07<00:01,  1.06it/s]

[10:35:06] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:08<00:00,  1.07it/s]

[10:35:07] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:09<00:00,  1.09it/s]


In [6]:
Wheeze_dataframe = pd.read_csv("./breath_csv/Wheeze.csv")
Wheeze_dataframe_1 = pd.read_csv("./breath_csv/Wheeze_1.csv")
Wheeze_dataframe_2 = pd.read_csv("./breath_csv/Wheeze_2.csv")
Wheeze_dataframe_3 = pd.read_csv("./breath_csv/Wheeze_3.csv")

In [50]:
make_legit_sets(Wheeze_dataframe, 'Wheeze')
make_legit_sets(Wheeze_dataframe_1, 'Wheeze_1')
make_legit_sets(Wheeze_dataframe_2, 'Wheeze_2')
make_legit_sets(Wheeze_dataframe_3, 'Wheeze_3')

In [51]:
MLA_Wheeze = MLA_selection(Wheeze_dataframe, 'Wheeze', "./breathe_set_csv/")
MLA_Wheeze_1 = MLA_selection(Wheeze_dataframe_1, 'Wheeze_1', "./breathe_set_csv/")
MLA_Wheeze_2 = MLA_selection(Wheeze_dataframe_2, 'Wheeze_2', "./breathe_set_csv/")
MLA_Wheeze_3 = MLA_selection(Wheeze_dataframe_3, 'Wheeze_3', "./breathe_set_csv/")

  0%|          | 0/10 [00:00<?, ?it/s]

[15:41:10] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:03<00:30,  3.43s/it]

[15:41:13] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:06<00:26,  3.32s/it]

...

 80%|████████  | 8/10 [00:25<00:06,  3.24s/it]

[15:45:54] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:29<00:03,  3.25s/it]

[15:45:57] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:32<00:00,  3.23s/it]


In [52]:
MLA_Wheeze

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9992805755395684, 1.0, 0.9992690058479532, ...",0.99971,"[0.8923766816143498, 0.8683127572016461, 0.759...",0.859796,0.0572516
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8923766816143498, 0.8559670781893004, 0.751...",0.857264,0.0582195
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9935251798561151, 0.9927007299270073, 0.991...",0.991896,"[0.8923766816143498, 0.8641975308641975, 0.726...",0.850744,0.0645768
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9784172661870504, 0.9766423357664233, 0.979...",0.974519,"[0.852017937219731, 0.8271604938271605, 0.7428...",0.817573,0.0435298
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8985611510791367, 0.910948905109489, 0.9210...",0.902301,"[0.8295964125560538, 0.7366255144032922, 0.702...",0.793896,0.0562414


In [53]:
MLA_Wheeze_1

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8340807174887892, 0.8148148148148148, 0.661...",0.789876,0.0796238
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9978417266187051, 1.0, 0.9985380116959064, ...",0.999565,"[0.8295964125560538, 0.7860082304526749, 0.665...",0.783063,0.0749197
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9741007194244604, 0.9766423357664233, 0.981...",0.97132,"[0.8026905829596412, 0.7695473251028807, 0.644...",0.775536,0.0743698
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9410071942446043, 0.9437956204379562, 0.952...",0.937517,"[0.8071748878923767, 0.7613168724279835, 0.673...",0.774306,0.0727206
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8597122302158273, 0.8751824817518248, 0.889...",0.867122,"[0.8161434977578476, 0.7366255144032922, 0.693...",0.767092,0.0588716


In [54]:
MLA_Wheeze_2

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.874439461883408, 0.831275720164609, 0.73877...",0.838803,0.0485347
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8834080717488789, 0.8395061728395061, 0.689...",0.836816,0.063978
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9913669064748202, 0.9905109489051095, 0.989...",0.989931,"[0.8878923766816144, 0.8353909465020576, 0.702...",0.831058,0.0622819
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9683453237410072, 0.9700729927007299, 0.974...",0.967453,"[0.8834080717488789, 0.8436213991769548, 0.714...",0.827548,0.0572437
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.89568345323741, 0.9, 0.9166666666666666, 0....",0.894649,"[0.8475336322869955, 0.7407407407407407, 0.673...",0.791801,0.0600646


In [55]:
MLA_Wheeze_3

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9517985611510792, 0.9518248175182482, 0.959...",0.951178,"[0.8475336322869955, 0.8024691358024691, 0.742...",0.795867,0.0364929
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8733812949640288, 0.8868613138686131, 0.907...",0.882832,"[0.8385650224215246, 0.7736625514403292, 0.681...",0.791925,0.0662885
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8565022421524664, 0.7695473251028807, 0.673...",0.789259,0.0538737
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9992805755395684, 0.9992700729927008, 0.999...",0.99949,"[0.852017937219731, 0.7736625514403292, 0.6530...",0.782801,0.0545316
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9669064748201439, 0.9773722627737226, 0.977...",0.973293,"[0.8609865470852018, 0.7654320987654321, 0.657...",0.7775,0.0546414


In [7]:
Wheeze_dataframe_0 = pd.read_csv("./breath_csv/Wheeze_0.csv")
Wheeze_dataframe_5 = pd.read_csv("./breath_csv/Wheeze_5.csv")
Wheeze_dataframe_10 = pd.read_csv("./breath_csv/Wheeze_10.csv")
Wheeze_dataframe_15 = pd.read_csv("./breath_csv/Wheeze_15.csv")
Wheeze_dataframe_20 = pd.read_csv("./breath_csv/Wheeze_20.csv")
Wheeze_dataframe_25 = pd.read_csv("./breath_csv/Wheeze_25.csv")
Wheeze_dataframe_30 = pd.read_csv("./breath_csv/Wheeze_30.csv")
Wheeze_dataframe_35 = pd.read_csv("./breath_csv/Wheeze_35.csv")
Wheeze_dataframe_40 = pd.read_csv("./breath_csv/Wheeze_40.csv")
Wheeze_dataframe_45 = pd.read_csv("./breath_csv/Wheeze_45.csv")
Wheeze_dataframe_50 = pd.read_csv("./breath_csv/Wheeze_50.csv")


In [57]:
make_legit_sets(Wheeze_dataframe_0, 'Wheeze_0')
make_legit_sets(Wheeze_dataframe_5, 'Wheeze_5')
make_legit_sets(Wheeze_dataframe_10, 'Wheeze_10')
make_legit_sets(Wheeze_dataframe_15, 'Wheeze_15')
make_legit_sets(Wheeze_dataframe_20, 'Wheeze_20')
make_legit_sets(Wheeze_dataframe_25, 'Wheeze_25')
make_legit_sets(Wheeze_dataframe_30, 'Wheeze_30')
make_legit_sets(Wheeze_dataframe_35, 'Wheeze_35')
make_legit_sets(Wheeze_dataframe_40, 'Wheeze_40')
make_legit_sets(Wheeze_dataframe_45, 'Wheeze_45')
make_legit_sets(Wheeze_dataframe_50, 'Wheeze_50')

In [10]:
MLA_wheeze_0 = MLA_selection(Wheeze_dataframe_0, 'Wheeze_0', "./breathe_set_csv/")
MLA_wheeze_5 = MLA_selection(Wheeze_dataframe_5, 'Wheeze_5', "./breathe_set_csv/")
MLA_wheeze_10 = MLA_selection(Wheeze_dataframe_10, 'Wheeze_10', "./breathe_set_csv/")
MLA_wheeze_15 = MLA_selection(Wheeze_dataframe_15, 'Wheeze_15', "./breathe_set_csv/")
MLA_wheeze_20 = MLA_selection(Wheeze_dataframe_20, 'Wheeze_20', "./breathe_set_csv/")
MLA_wheeze_25 = MLA_selection(Wheeze_dataframe_25, 'Wheeze_25', "./breathe_set_csv/")
MLA_wheeze_30 = MLA_selection(Wheeze_dataframe_30, 'Wheeze_30', "./breathe_set_csv/")
MLA_wheeze_35 = MLA_selection(Wheeze_dataframe_35, 'Wheeze_35', "./breathe_set_csv/")
MLA_wheeze_40 = MLA_selection(Wheeze_dataframe_40, 'Wheeze_40', "./breathe_set_csv/")
MLA_wheeze_45 = MLA_selection(Wheeze_dataframe_45, 'Wheeze_45', "./breathe_set_csv/")
MLA_wheeze_50 = MLA_selection(Wheeze_dataframe_50, 'Wheeze_50', "./breathe_set_csv/")

  0%|          | 0/10 [00:00<?, ?it/s]

[06:58:44] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:06<00:57,  6.35s/it]

[06:58:49] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:13<00:53,  6.68s/it]

...

 80%|████████  | 8/10 [00:27<00:06,  3.50s/it]

[07:14:10] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:31<00:03,  3.51s/it]

[07:14:14] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:34<00:00,  3.45s/it]


In [32]:
MLA_wheeze_0

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8787878787878788, 0.8145161290322581, 0.677...",0.81201,0.0731933
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9881656804733728, 0.9917602996254682, 0.991...",0.987607,"[0.8614718614718615, 0.7903225806451613, 0.681...",0.803733,0.0738866
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9992603550295858, 1.0, 0.9992542878448919, ...",0.999478,"[0.8744588744588745, 0.7903225806451613, 0.690...",0.800568,0.0717622
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9578402366863905, 0.9588014981273408, 0.964...",0.955128,"[0.8484848484848485, 0.7862903225806451, 0.677...",0.798212,0.0665819
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8816568047337278, 0.8913857677902621, 0.902...",0.884447,"[0.8658008658008658, 0.7661290322580645, 0.706...",0.789555,0.0754782


In [33]:
MLA_wheeze_5

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8654708520179372, 0.8189300411522634, 0.697...",0.822651,0.0688491
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9892086330935251, 0.983941605839416, 0.9948...",0.987374,"[0.8699551569506726, 0.7818930041152263, 0.673...",0.812549,0.0692953
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9985611510791367, 1.0, 0.9992690058479532, ...",0.999564,"[0.874439461883408, 0.7777777777777778, 0.6693...",0.809227,0.0706118
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8949640287769784, 0.9007299270072993, 0.910...",0.892648,"[0.8565022421524664, 0.7654320987654321, 0.706...",0.802857,0.0582784
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9568345323741008, 0.9554744525547445, 0.966...",0.953509,"[0.852017937219731, 0.7860082304526749, 0.6816...",0.800709,0.0710782


In [34]:
MLA_wheeze_10

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9992805755395684, 0.9992700729927008, 0.999...",0.999416,"[0.8789237668161435, 0.8436213991769548, 0.706...",0.83244,0.0662991
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9906474820143885, 0.9897810218978103, 0.991...",0.987295,"[0.8654708520179372, 0.8271604938271605, 0.710...",0.824802,0.0574132
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8654708520179372, 0.8106995884773662, 0.677...",0.822579,0.0685646
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9503597122302159, 0.9554744525547445, 0.967...",0.951614,"[0.852017937219731, 0.8148148148148148, 0.7020...",0.810737,0.0671203
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8913669064748202, 0.8934306569343066, 0.904...",0.887183,"[0.8475336322869955, 0.7283950617283951, 0.718...",0.796493,0.0561591


In [35]:
MLA_wheeze_15

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9992805755395684, 1.0, 0.9992690058479532, ...",0.999783,"[0.8878923766816144, 0.8477366255144033, 0.734...",0.841553,0.0549448
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8834080717488789, 0.8395061728395061, 0.730...",0.839555,0.0603145
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9870503597122302, 0.9883211678832117, 0.986...",0.986289,"[0.8789237668161435, 0.8395061728395061, 0.718...",0.829652,0.0572438
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.958273381294964, 0.9613138686131387, 0.9649...",0.957451,"[0.852017937219731, 0.8189300411522634, 0.6979...",0.816277,0.0602079
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8942446043165467, 0.897080291970803, 0.9064...",0.891054,"[0.8340807174887892, 0.757201646090535, 0.7224...",0.802638,0.0484924


In [36]:
MLA_wheeze_20

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 0.9992690058479532, 1.0, 1.0, 1.0, ...",0.999927,"[0.8923766816143498, 0.8559670781893004, 0.718...",0.848103,0.0557233
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8789237668161435, 0.8477366255144033, 0.751...",0.84467,0.0550096
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9913669064748202, 0.9919708029197081, 0.992...",0.991393,"[0.8789237668161435, 0.8353909465020576, 0.722...",0.841494,0.0562063
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9597122302158273, 0.9642335766423358, 0.968...",0.960664,"[0.852017937219731, 0.823045267489712, 0.70612...",0.820056,0.0549133
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.9043165467625899, 0.8978102189781022, 0.904...",0.890537,"[0.8385650224215246, 0.7530864197530864, 0.702...",0.79522,0.0563779


In [37]:
MLA_wheeze_25

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.874439461883408, 0.831275720164609, 0.73877...",0.838803,0.0485347
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8968609865470852, 0.8559670781893004, 0.693...",0.838007,0.0638339
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9913669064748202, 0.9905109489051095, 0.989...",0.989931,"[0.8878923766816144, 0.8353909465020576, 0.702...",0.831475,0.0628644
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9683453237410072, 0.9700729927007299, 0.974...",0.967453,"[0.8834080717488789, 0.8436213991769548, 0.714...",0.827548,0.0572437
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8942446043165467, 0.9007299270072993, 0.916...",0.894354,"[0.8385650224215246, 0.7407407407407407, 0.673...",0.790904,0.0592877


In [38]:
MLA_wheeze_30

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.874439461883408, 0.8395061728395061, 0.6979...",0.835578,0.0595365
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9827338129496402, 0.9897810218978103, 0.989...",0.988345,"[0.8654708520179372, 0.831275720164609, 0.7224...",0.828534,0.0619933
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.999...",0.999927,"[0.874439461883408, 0.8477366255144033, 0.7061...",0.828478,0.0595824
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9705035971223022, 0.9744525547445255, 0.977...",0.970009,"[0.8654708520179372, 0.8395061728395061, 0.722...",0.826189,0.0568804
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8949640287769784, 0.9021897810218978, 0.915...",0.896327,"[0.8475336322869955, 0.7489711934156379, 0.665...",0.800391,0.067259


In [39]:
MLA_wheeze_35

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9992700729927008, 0.9992690058479532, ...",0.999709,"[0.8699551569506726, 0.8477366255144033, 0.722...",0.831974,0.0539007
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9841726618705036, 0.9883211678832117, 0.989...",0.984457,"[0.8789237668161435, 0.8436213991769548, 0.710...",0.827077,0.0629357
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8654708520179372, 0.8477366255144033, 0.697...",0.825008,0.0592232
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9683453237410072, 0.9708029197080292, 0.975...",0.968613,"[0.8654708520179372, 0.823045267489712, 0.7265...",0.816833,0.0542838
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8928057553956834, 0.9036496350364963, 0.913...",0.89646,"[0.8430493273542601, 0.7736625514403292, 0.681...",0.797827,0.0616497


In [40]:
MLA_wheeze_40

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9985401459854014, 0.9992690058479532, ...",0.999415,"[0.8878923766816144, 0.8395061728395061, 0.685...",0.827594,0.0591697
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9863309352517986, 0.9868613138686131, 0.983...",0.983212,"[0.8699551569506726, 0.831275720164609, 0.6979...",0.825722,0.0581908
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.874439461883408, 0.823045267489712, 0.70204...",0.823759,0.0604517
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9633093525179857, 0.9686131386861314, 0.970...",0.965184,"[0.852017937219731, 0.831275720164609, 0.73469...",0.810145,0.0477686
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8884892086330936, 0.8905109489051095, 0.906...",0.887781,"[0.8565022421524664, 0.757201646090535, 0.7061...",0.792174,0.0601467


In [41]:
MLA_wheeze_45

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8475336322869955, 0.8353909465020576, 0.677...",0.808833,0.0533822
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9992700729927008, 0.9992690058479532, ...",0.999781,"[0.8609865470852018, 0.8148148148148148, 0.665...",0.807297,0.0639296
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9733812949640288, 0.9890510948905109, 0.990...",0.980886,"[0.8654708520179372, 0.8148148148148148, 0.681...",0.804254,0.0605905
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9661870503597122, 0.9700729927007299, 0.972...",0.965841,"[0.8295964125560538, 0.8106995884773662, 0.706...",0.794744,0.0497255
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8820143884892087, 0.8883211678832117, 0.906...",0.885237,"[0.8340807174887892, 0.7283950617283951, 0.697...",0.775051,0.0600383


In [42]:
MLA_wheeze_50

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8565022421524664, 0.8024691358024691, 0.706...",0.810586,0.0495225
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.981294964028777, 0.983941605839416, 0.98976...",0.982128,"[0.874439461883408, 0.8024691358024691, 0.6897...",0.807297,0.0520809
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.852017937219731, 0.7942386831275721, 0.6734...",0.8066,0.0587598
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9676258992805755, 0.9693430656934306, 0.973...",0.966575,"[0.8251121076233184, 0.8106995884773662, 0.755...",0.803454,0.045834
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8848920863309353, 0.8854014598540146, 0.904...",0.88934,"[0.8251121076233184, 0.7448559670781894, 0.685...",0.780059,0.0638674


## Breath(258)

In [123]:
Total = np.unique(os.listdir("./breathe_mfcc/"))
name = []
for i in Total:
    name.append(i[:4])

unique = []
before_after = []
for i in Total:
    if np.sum((np.array(name, dtype = int) == int(i[:4]))) == 1:
        unique.append(i)
    else:
        before_after.append(i) 

Asthmatic_Female = []
Asthmatic_Male = []
Healthy_Male = []
Healthy_Female = []
for file in unique:
    if file.find('sthma') !=-1:
        if file.find("_M_")!=-1:
            Asthmatic_Male.append(file)
        if file.find("_F_")!=-1:
            Asthmatic_Female.append(file)
    if file.find("_C_")!=-1:
        if file.find("_M_")!=-1:
            Healthy_Male.append(file)
        if file.find("_F_")!=-1:
            Healthy_Female.append(file)
            
print(f"Total = {len(Total)}")
print(f"Asthmatic_male = {len(Asthmatic_Male)}")
print(f"Asthmatic_female = {len(Asthmatic_Female)}")
print(f"Healthy_Male = {len(Healthy_Male)}")
print(f"Healthy_female = {len(Healthy_Female)}")
print(f"Before_after = {len(before_after)}")

Total = 258
Asthmatic_male = 78
Asthmatic_female = 55
Healthy_Male = 55
Healthy_female = 50
Before_after = 20


In [128]:
Na = []
for i in (Total):
    if i.find("NAsthma") != -1:
        Na.append(i)

In [130]:
len(Na)

9

In [135]:
before_after

['2135_P_Asthma_F_before_Sev_Y.json',
 '2135_P_NAsthma_F_before_TBD_N.json',
 '2143_P_Asthma_F_after_TBD_Y.json',
 '2143_P_NAsthma_M_before_TBD_Y.json',
 '2172_P_Asthma_F_before_Sev_Y.json',
 '2172_P_Asthma_M_before_Sev_Y.json',
 '2205_P_Asthma_F_after_Mod_Y.json',
 '2205_P_Asthma_F_after_Sev_Y.json',
 '2242_P_Asthma_F_before_TBD_N.json',
 '2242_Reddy_P_Asthma_M_Sev_Y.json',
 '2296_P_Asthma_F_before_Mod_Y.json',
 '2296_P_Asthma_F_before_TBD_Y.json',
 '2365_P_Asthma_F_after_TBD_Y.json',
 '2365_P_Asthma_M_before_Mil_Y.json',
 '2372_P_Asthma_F_before_Mil_Y.json',
 '2372_P_NAsthma_M_After_Na_Y.json',
 '2394_P_Asthma_M_after_Sev_Y.json',
 '2394_P_Asthma_M_before_Mod_Y.json',
 '2499_P_Asthma_F_before_TBD_Y.json',
 '2499_P_Asthma_M_after_Mil_Y.json']

In [80]:
breath_dataframe = pd.read_csv("./breath_csv/Breath.csv")
breath_dataframe_1 = pd.read_csv("./breath_csv/Breath_1.csv")
breath_dataframe_2 = pd.read_csv("./breath_csv/Breath_2.csv")
breath_dataframe_3 = pd.read_csv("./breath_csv/Breath_3.csv")

In [81]:
make_legit_sets(breath_dataframe, 'breath', "breath_sets")
make_legit_sets(breath_dataframe_1, 'breath_1', "breath_sets")
make_legit_sets(breath_dataframe_2, 'breath_2', "breath_sets")
make_legit_sets(breath_dataframe_3, 'breath_3', "breath_sets")

In [82]:
MLA_breath = MLA_selection(breath_dataframe, 'breath', "./breathe_set_csv/")
MLA_breath_1 = MLA_selection(breath_dataframe_1, 'breath_1', "./breathe_set_csv/")
MLA_breath_2 = MLA_selection(breath_dataframe_2, 'breath_2', "./breathe_set_csv/")
MLA_breath_3 = MLA_selection(breath_dataframe_3, 'breath_3', "./breathe_set_csv/")

  0%|          | 0/10 [00:00<?, ?it/s]

[10:56:04] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:07<01:06,  7.35s/it]

[10:56:12] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:14<01:00,  7.52s/it]

...

 80%|████████  | 8/10 [00:44<00:10,  5.23s/it]

[11:04:58] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:49<00:05,  5.19s/it]

[11:05:03] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:54<00:00,  5.44s/it]


In [89]:
MLA_breath

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9990272373540856, 0.9995147986414362, 1.0, ...",0.999611,"[0.744, 0.8163265306122449, 0.7609561752988048...",0.789657,0.0420784
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.72, 0.8326530612244898, 0.7808764940239044,...",0.787837,0.0439515
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9644941634241245, 0.9631246967491509, 0.961...",0.962044,"[0.728, 0.8489795918367347, 0.7768924302788844...",0.786593,0.0491957
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8244163424124513, 0.8204754973313926, 0.836...",0.8289,"[0.716, 0.8571428571428571, 0.7051792828685259...",0.771672,0.0616491
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9591439688715954, 0.9621542940320232, 0.954...",0.957952,"[0.712, 0.8, 0.7211155378486056, 0.72608695652...",0.76079,0.049232


In [90]:
MLA_breath_1

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9990272373540856, 1.0, 0.9995133819951338, ...",0.999611,"[0.732, 0.8489795918367347, 0.7649402390438247...",0.804653,0.0505609
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.716, 0.8040816326530612, 0.7649402390438247...",0.786326,0.0457245
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9698443579766537, 0.9621542940320232, 0.966...",0.96229,"[0.72, 0.8244897959183674, 0.7649402390438247,...",0.78453,0.0480982
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.960603112840467, 0.9568170790878214, 0.9537...",0.957855,"[0.716, 0.8285714285714286, 0.7450199203187251...",0.760947,0.0506081
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8234435797665369, 0.8204754973313926, 0.837...",0.826995,"[0.688, 0.8448979591836735, 0.6772908366533864...",0.758223,0.066867


In [91]:
MLA_breath_2

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9995083579154...",0.999902,"[0.748, 0.8653061224489796, 0.749003984063745,...",0.799478,0.0476157
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.74, 0.8244897959183674, 0.749003984063745, ...",0.789791,0.0422323
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9693579766536965, 0.9631246967491509, 0.965...",0.961866,"[0.748, 0.8489795918367347, 0.7410358565737052...",0.781597,0.046424
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8351167315175098, 0.8296943231441049, 0.841...",0.835491,"[0.74, 0.8612244897959184, 0.701195219123506, ...",0.77403,0.0558266
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9508754863813229, 0.9514798641436196, 0.950...",0.951907,"[0.74, 0.8, 0.7051792828685259, 0.769565217391...",0.767594,0.0499834


In [92]:
MLA_breath_3

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8180933852140078, 0.8073750606501698, 0.827...",0.81803,"[0.7, 0.8204081632653061, 0.6892430278884463, ...",0.758578,0.0455179
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9212062256809338, 0.9170305676855895, 0.919...",0.919081,"[0.716, 0.763265306122449, 0.7171314741035857,...",0.738738,0.0302361
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9294747081712063, 0.9277049975739932, 0.923...",0.927715,"[0.68, 0.8081632653061225, 0.7131474103585658,...",0.729069,0.043369
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.712, 0.7795918367346939, 0.7171314741035857...",0.727326,0.0438597
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9995136186770428, 0.9985443959243085, 1.0, ...",0.99927,"[0.712, 0.7673469387755102, 0.7091633466135459...",0.726228,0.0345637


In [83]:
breath_dataframe_0 = pd.read_csv("./breath_csv/Breath_0.csv")
breath_dataframe_5 = pd.read_csv("./breath_csv/Breath_5.csv")
breath_dataframe_10 = pd.read_csv("./breath_csv/Breath_10.csv")
breath_dataframe_15 = pd.read_csv("./breath_csv/Breath_15.csv")
breath_dataframe_20 = pd.read_csv("./breath_csv/Breath_20.csv")
breath_dataframe_25 = pd.read_csv("./breath_csv/Breath_25.csv")
breath_dataframe_30 = pd.read_csv("./breath_csv/Breath_30.csv")
breath_dataframe_35 = pd.read_csv("./breath_csv/Breath_35.csv")
breath_dataframe_40 = pd.read_csv("./breath_csv/Breath_40.csv")
breath_dataframe_45 = pd.read_csv("./breath_csv/Breath_45.csv")
breath_dataframe_50 = pd.read_csv("./breath_csv/Breath_50.csv")


make_legit_sets(breath_dataframe_0, 'breath_0', "breath_sets")
make_legit_sets(breath_dataframe_5, 'breath_5', "breath_sets")
make_legit_sets(breath_dataframe_10, 'breath_10', "breath_sets")
make_legit_sets(breath_dataframe_15, 'breath_15', "breath_sets")
make_legit_sets(breath_dataframe_20, 'breath_20', "breath_sets")
make_legit_sets(breath_dataframe_25, 'breath_25', "breath_sets")
make_legit_sets(breath_dataframe_30, 'breath_30', "breath_sets")
make_legit_sets(breath_dataframe_35, 'breath_35', "breath_sets")
make_legit_sets(breath_dataframe_40, 'breath_40', "breath_sets")
make_legit_sets(breath_dataframe_45, 'breath_45', "breath_sets")
make_legit_sets(breath_dataframe_50, 'breath_50', "breath_sets")


MLA_breath_0 = MLA_selection(breath_dataframe_0, 'breath_0', "./breathe_set_csv/")
MLA_breath_5 = MLA_selection(breath_dataframe_5, 'breath_5', "./breathe_set_csv/")
MLA_breath_10 = MLA_selection(breath_dataframe_10, 'breath_10', "./breathe_set_csv/")
MLA_breath_15 = MLA_selection(breath_dataframe_15, 'breath_15', "./breathe_set_csv/")
MLA_breath_20 = MLA_selection(breath_dataframe_20, 'breath_20', "./breathe_set_csv/")
MLA_breath_25 = MLA_selection(breath_dataframe_25, 'breath_25', "./breathe_set_csv/")
MLA_breath_30 = MLA_selection(breath_dataframe_30, 'breath_30', "./breathe_set_csv/")
MLA_breath_35 = MLA_selection(breath_dataframe_35, 'breath_35', "./breathe_set_csv/")
MLA_breath_40 = MLA_selection(breath_dataframe_40, 'breath_40', "./breathe_set_csv/")
MLA_breath_45 = MLA_selection(breath_dataframe_45, 'breath_45', "./breathe_set_csv/")
MLA_breath_50 = MLA_selection(breath_dataframe_50, 'breath_50', "./breathe_set_csv/")

  0%|          | 0/10 [00:00<?, ?it/s]

[11:08:08] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:13<02:02, 13.58s/it]

[11:08:21] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:24<01:35, 11.90s/it]

...

 80%|████████  | 8/10 [01:07<00:18,  9.08s/it]

[11:34:18] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [01:22<00:10, 10.75s/it]

[11:34:32] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [01:28<00:00,  8.84s/it]


In [93]:
MLA_breath_0

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.676, 0.8081632653061225, 0.7370517928286853...",0.757534,0.0555135
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9995083579154...",0.999902,"[0.684, 0.7918367346938775, 0.749003984063745,...",0.755091,0.0556428
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9523346303501945, 0.9378942261038331, 0.949...",0.947148,"[0.684, 0.7918367346938775, 0.7091633466135459...",0.75289,0.051439
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9246108949416343, 0.9189713731198448, 0.924...",0.922305,"[0.7, 0.7795918367346939, 0.7091633466135459, ...",0.748088,0.0404865
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8098249027237354, 0.7976710334788938, 0.813...",0.805487,"[0.664, 0.7877551020408163, 0.6653386454183267...",0.728433,0.0577682


In [94]:
MLA_breath_5

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7, 0.7836734693877551, 0.7290836653386454, ...",0.765185,0.0475778
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9995147986414362, 1.0, 0.9995183044315...",0.99961,"[0.692, 0.8040816326530612, 0.7450199203187251...",0.760422,0.0544071
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9464980544747081, 0.9446870451237264, 0.951...",0.948217,"[0.668, 0.7918367346938775, 0.7171314741035857...",0.754387,0.0531561
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9236381322957199, 0.9170305676855895, 0.925...",0.921084,"[0.68, 0.8, 0.7051792828685259, 0.743478260869...",0.741233,0.0436627
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.811284046692607, 0.7981562348374576, 0.8063...",0.805399,"[0.692, 0.7714285714285715, 0.6693227091633466...",0.723191,0.039671


In [95]:
MLA_breath_10

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9995136186770428, 1.0, 0.9995133819951338, ...",0.999756,"[0.72, 0.7551020408163265, 0.749003984063745, ...",0.769016,0.0495016
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9523346303501945, 0.9432314410480349, 0.948...",0.946003,"[0.712, 0.7673469387755102, 0.7250996015936255...",0.765019,0.0465541
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.708, 0.7755102040816326, 0.7211155378486056...",0.761079,0.0456155
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9231517509727627, 0.9247937894226104, 0.923...",0.92333,"[0.676, 0.7959183673469388, 0.7250996015936255...",0.743149,0.0437229
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8035019455252919, 0.7976710334788938, 0.807...",0.801196,"[0.676, 0.7959183673469388, 0.6932270916334662...",0.739934,0.0448449


In [96]:
MLA_breath_15

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.732, 0.8163265306122449, 0.7051792828685259...",0.771343,0.0511345
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9995083579154...",0.999951,"[0.728, 0.7877551020408163, 0.7290836653386454...",0.770229,0.0498149
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9455252918287937, 0.9325570111596313, 0.941...",0.942122,"[0.664, 0.7591836734693878, 0.701195219123506,...",0.752044,0.0609641
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9294747081712063, 0.9311014070839398, 0.935...",0.931518,"[0.736, 0.7836734693877551, 0.7529880478087649...",0.745672,0.0474915
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8103112840466926, 0.8049490538573508, 0.811...",0.80523,"[0.68, 0.8163265306122449, 0.6772908366533864,...",0.742079,0.0566886


In [97]:
MLA_breath_20

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.74, 0.8040816326530612, 0.7171314741035857,...",0.772475,0.0436979
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.744, 0.8285714285714286, 0.7051792828685259...",0.770514,0.0445879
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9469844357976653, 0.9393498301795244, 0.946...",0.946217,"[0.724, 0.7959183673469388, 0.7091633466135459...",0.769509,0.0547009
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9323929961089494, 0.9340126152353226, 0.934...",0.933713,"[0.732, 0.7755102040816326, 0.7290836653386454...",0.753108,0.0380417
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8205252918287937, 0.8064046579330422, 0.824...",0.814463,"[0.664, 0.8163265306122449, 0.6374501992031872...",0.745124,0.068183


In [98]:
MLA_breath_25

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.736, 0.8530612244897959, 0.7051792828685259...",0.767874,0.0456826
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8171206225680934, 0.8107714701601164, 0.835...",0.819939,"[0.732, 0.8285714285714286, 0.6693227091633466...",0.767219,0.0552064
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.712, 0.8081632653061225, 0.7171314741035857...",0.766145,0.0425315
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9513618677042801, 0.9417758369723436, 0.950...",0.946389,"[0.712, 0.8040816326530612, 0.7091633466135459...",0.759243,0.047283
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.936284046692607, 0.9369238233867054, 0.9401...",0.938641,"[0.744, 0.8040816326530612, 0.7131474103585658...",0.754245,0.0435252


In [99]:
MLA_breath_30

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8176070038910506, 0.8107714701601164, 0.836...",0.820376,"[0.736, 0.8530612244897959, 0.6812749003984063...",0.770629,0.0520552
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 0.9995183044315993, 1.0, 1.0, ...",0.999952,"[0.712, 0.8367346938775511, 0.7370517928286853...",0.770393,0.0405897
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9499027237354085, 0.9301310043668122, 0.941...",0.945485,"[0.716, 0.8285714285714286, 0.7250996015936255...",0.76232,0.0470762
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9372568093385214, 0.9378942261038331, 0.938...",0.938639,"[0.724, 0.8204081632653061, 0.7091633466135459...",0.759914,0.0458842
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7, 0.8326530612244898, 0.7091633466135459, ...",0.752993,0.0469297


In [100]:
MLA_breath_35

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9995136186770428, 0.9995147986414362, 0.999...",0.999611,"[0.716, 0.8367346938775511, 0.7450199203187251...",0.768832,0.0371253
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9392023346303502, 0.9442018437651626, 0.939...",0.94074,"[0.708, 0.8285714285714286, 0.7051792828685259...",0.759189,0.0505117
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.708, 0.8244897959183674, 0.7211155378486056...",0.755813,0.0372315
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9484435797665369, 0.9359534206695779, 0.943...",0.942314,"[0.712, 0.8122448979591836, 0.7330677290836654...",0.755165,0.0438386
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8180933852140078, 0.8030082484230956, 0.827...",0.814856,"[0.688, 0.8612244897959184, 0.6374501992031872...",0.752857,0.0665143


In [101]:
MLA_breath_40

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9995136186770428, 0.9985443959243085, 0.999...",0.999269,"[0.724, 0.8204081632653061, 0.7370517928286853...",0.762313,0.0337721
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9542801556420234, 0.9442018437651626, 0.945...",0.945911,"[0.704, 0.8122448979591836, 0.7051792828685259...",0.759296,0.041722
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.704, 0.7877551020408163, 0.6972111553784861...",0.746506,0.0311844
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.943579766536965, 0.9432314410480349, 0.9445...",0.942539,"[0.708, 0.8122448979591836, 0.6972111553784861...",0.745676,0.051594
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.808852140077821, 0.8068898592916061, 0.8277...",0.812555,"[0.684, 0.8204081632653061, 0.6135458167330677...",0.732723,0.064255


In [102]:
MLA_breath_45

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9995136186770428, 0.9995147986414362, 0.999...",0.999074,"[0.728, 0.8081632653061225, 0.7051792828685259...",0.754918,0.0331223
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.72, 0.8163265306122449, 0.7091633466135459,...",0.753127,0.0368635
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9469844357976653, 0.9359534206695779, 0.935...",0.9384,"[0.728, 0.8122448979591836, 0.6812749003984063...",0.749083,0.0402679
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8146887159533074, 0.8093158660844251, 0.830...",0.815575,"[0.68, 0.8612244897959184, 0.6254980079681275,...",0.743659,0.0656314
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9450389105058365, 0.9456574478408539, 0.946...",0.94532,"[0.68, 0.8040816326530612, 0.6972111553784861,...",0.729897,0.0464441


In [103]:
MLA_breath_50

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9995136186770428, 0.9995147986414362, 0.999...",0.999318,"[0.748, 0.8040816326530612, 0.7171314741035857...",0.762051,0.0277347
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9450389105058365, 0.9344978165938864, 0.938...",0.938602,"[0.736, 0.7959183673469388, 0.7051792828685259...",0.760469,0.0311832
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8214980544747081, 0.8102862688015526, 0.828...",0.819295,"[0.688, 0.8326530612244898, 0.6573705179282868...",0.74948,0.0538932
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.684, 0.8163265306122449, 0.6852589641434262...",0.743647,0.0399432
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9430933852140078, 0.9475982532751092, 0.943...",0.943557,"[0.696, 0.7836734693877551, 0.6972111553784861...",0.741902,0.0407612


### Random 188

In [84]:
Total = np.unique(pd.read_csv("./Random_188.txt", header=None)[0].values)
name = []
for i in Total:
    name.append(i[:4])

unique = []
before_after = []
for i in Total:
    if np.sum((np.array(name, dtype = int) == int(i[:4]))) == 1:
        unique.append(i)
    else:
        before_after.append(i) 

Asthmatic_Female = []
Asthmatic_Male = []
Healthy_Male = []
Healthy_Female = []
for file in unique:
    if file.find('sthma') !=-1:
        if file.find("_M_")!=-1:
            Asthmatic_Male.append(file)
        if file.find("_F_")!=-1:
            Asthmatic_Female.append(file)
    if file.find("_C_")!=-1:
        if file.find("_M_")!=-1:
            Healthy_Male.append(file)
        if file.find("_F_")!=-1:
            Healthy_Female.append(file)
            
print(f"Total = {len(Total)}")
print(f"Asthmatic_male = {len(Asthmatic_Male)}")
print(f"Asthmatic_female = {len(Asthmatic_Female)}")
print(f"Healthy_Male = {len(Healthy_Male)}")
print(f"Healthy_female = {len(Healthy_Female)}")
print(f"Before_after = {len(before_after)}")

Total = 188
Asthmatic_male = 57
Asthmatic_female = 45
Healthy_Male = 42
Healthy_female = 38
Before_after = 6


In [85]:
def b_r(sound_dataframe, total):
    test_dataframe = pd.DataFrame()
    for i, line in enumerate(total):
        A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (total[i]))]
        test_dataframe = pd.DataFrame.append(test_dataframe,A)
    return test_dataframe


In [86]:
random_breath_dataframe_0 =b_r( breath_dataframe_0, Total)
random_breath_dataframe_5 =b_r( breath_dataframe_5, Total)
random_breath_dataframe_10 = b_r(breath_dataframe_10, Total)
random_breath_dataframe_15 = b_r(breath_dataframe_15, Total)
random_breath_dataframe_20 = b_r(breath_dataframe_20, Total)
random_breath_dataframe_25 = b_r(breath_dataframe_25, Total)
random_breath_dataframe_30 = b_r(breath_dataframe_30, Total)
random_breath_dataframe_35 = b_r(breath_dataframe_35, Total)
random_breath_dataframe_40 = b_r(breath_dataframe_40, Total)
random_breath_dataframe_45 = b_r(breath_dataframe_45, Total)
random_breath_dataframe_50 = b_r(breath_dataframe_50, Total)

In [87]:
make_legit_sets(random_breath_dataframe_0, 'random_breath_0', "Random_188_sets")
make_legit_sets(random_breath_dataframe_5, 'random_breath_5', "Random_188_sets")
make_legit_sets(random_breath_dataframe_10, 'random_breath_10', "Random_188_sets")
make_legit_sets(random_breath_dataframe_15, 'random_breath_15', "Random_188_sets")
make_legit_sets(random_breath_dataframe_20, 'random_breath_20', "Random_188_sets")
make_legit_sets(random_breath_dataframe_25, 'random_breath_25', "Random_188_sets")
make_legit_sets(random_breath_dataframe_30, 'random_breath_30', "Random_188_sets")
make_legit_sets(random_breath_dataframe_35, 'random_breath_35', "Random_188_sets")
make_legit_sets(random_breath_dataframe_40, 'random_breath_40', "Random_188_sets")
make_legit_sets(random_breath_dataframe_45, 'random_breath_45', "Random_188_sets")
make_legit_sets(random_breath_dataframe_50, 'random_breath_50', "Random_188_sets")


MLA_random_breath_0 = MLA_selection(random_breath_dataframe_0, 'random_breath_0', "./breathe_set_csv/")
MLA_random_breath_5 = MLA_selection(random_breath_dataframe_5, 'random_breath_5', "./breathe_set_csv/")
MLA_random_breath_10 = MLA_selection(random_breath_dataframe_10, 'random_breath_10', "./breathe_set_csv/")
MLA_random_breath_15 = MLA_selection(random_breath_dataframe_15, 'random_breath_15', "./breathe_set_csv/")
MLA_random_breath_20 = MLA_selection(random_breath_dataframe_20, 'random_breath_20', "./breathe_set_csv/")
MLA_random_breath_25 = MLA_selection(random_breath_dataframe_25, 'random_breath_25', "./breathe_set_csv/")
MLA_random_breath_30 = MLA_selection(random_breath_dataframe_30, 'random_breath_30', "./breathe_set_csv/")
MLA_random_breath_35 = MLA_selection(random_breath_dataframe_35, 'random_breath_35', "./breathe_set_csv/")
MLA_random_breath_40 = MLA_selection(random_breath_dataframe_40, 'random_breath_40', "./breathe_set_csv/")
MLA_random_breath_45 = MLA_selection(random_breath_dataframe_45, 'random_breath_45', "./breathe_set_csv/")
MLA_random_breath_50 = MLA_selection(random_breath_dataframe_50, 'random_breath_50', "./breathe_set_csv/")

  0%|          | 0/10 [00:00<?, ?it/s]

[11:36:37] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:03<00:30,  3.42s/it]

[11:36:41] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:06<00:27,  3.38s/it]

...

 80%|████████  | 8/10 [00:28<00:06,  3.44s/it]

[11:52:22] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:32<00:03,  3.78s/it]

[11:52:27] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:35<00:00,  3.60s/it]


In [104]:
MLA_random_breath_0

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.949375866851595, 0.9446778711484594, 0.9430...",0.943711,"[0.7553648068669528, 0.7327935222672065, 0.845...",0.781175,0.0477199
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7424892703862661, 0.7368421052631579, 0.845...",0.780997,0.0656891
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9819694868238558, 0.9754901960784313, 0.973...",0.976201,"[0.7381974248927039, 0.728744939271255, 0.8452...",0.769428,0.0600312
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 0.9992972593113141, 0.9985935302390...",0.999369,"[0.776824034334764, 0.7449392712550608, 0.8452...",0.769205,0.0625401
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8391123439667129, 0.8417366946778712, 0.833...",0.838411,"[0.7124463519313304, 0.7004048582995951, 0.773...",0.748138,0.0501143


In [105]:
MLA_random_breath_5

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7467811158798283, 0.7692307692307693, 0.853...",0.783569,0.0633542
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9993065187239945, 0.9992997198879552, 0.999...",0.9993,"[0.776824034334764, 0.7611336032388664, 0.8650...",0.777734,0.07195
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9507628294036061, 0.9474789915966386, 0.938...",0.943078,"[0.7510729613733905, 0.6963562753036437, 0.833...",0.77703,0.047149
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9798890429958391, 0.9740896358543417, 0.972...",0.974384,"[0.7253218884120172, 0.7206477732793523, 0.833...",0.754465,0.0633045
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.841886269070735, 0.8522408963585434, 0.8341...",0.841884,"[0.7253218884120172, 0.6923076923076923, 0.777...",0.747438,0.0532862


In [106]:
MLA_random_breath_10

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 0.9992972593113141, 1.0, 1.0, 0.999...",0.999717,"[0.7939914163090128, 0.7813765182186235, 0.880...",0.781628,0.0675741
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7639484978540773, 0.7611336032388664, 0.896...",0.780949,0.0706387
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9826629680998613, 0.9656862745098039, 0.973...",0.976013,"[0.7167381974248928, 0.7368421052631579, 0.849...",0.769752,0.0709112
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9438280166435506, 0.9432773109243697, 0.935...",0.943082,"[0.7253218884120172, 0.7206477732793523, 0.821...",0.769706,0.0571851
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8509015256588072, 0.8543417366946778, 0.846...",0.84804,"[0.7253218884120172, 0.708502024291498, 0.8015...",0.755909,0.0627034


In [107]:
MLA_random_breath_15

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7896995708154506, 0.7773279352226721, 0.853...",0.787208,0.0561659
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.999929,"[0.7939914163090128, 0.7975708502024291, 0.873...",0.786405,0.054306
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9466019417475728, 0.9502801120448179, 0.942...",0.946309,"[0.7339055793991416, 0.7206477732793523, 0.833...",0.775496,0.0552716
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9819694868238558, 0.976890756302521, 0.9789...",0.980064,"[0.7553648068669528, 0.7692307692307693, 0.837...",0.771663,0.0514766
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8411927877947295, 0.8529411764705882, 0.848...",0.850433,"[0.7381974248927039, 0.7206477732793523, 0.805...",0.755819,0.0535077


In [108]:
MLA_random_breath_20

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7639484978540773, 0.8137651821862348, 0.857...",0.787982,0.0495913
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.759656652360515, 0.7935222672064778, 0.8134...",0.781751,0.0426623
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9549237170596394, 0.9537815126050421, 0.945...",0.95352,"[0.7639484978540773, 0.728744939271255, 0.8293...",0.774999,0.050109
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8571428571428571, 0.8606442577030813, 0.853...",0.856621,"[0.7682403433476395, 0.7125506072874493, 0.813...",0.769679,0.0505461
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9826629680998613, 0.9684873949579832, 0.976...",0.980968,"[0.7339055793991416, 0.7692307692307693, 0.817...",0.763295,0.055761


In [109]:
MLA_random_breath_25

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.776824034334764, 0.8380566801619433, 0.8650...",0.792902,0.049579
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7682403433476395, 0.7854251012145749, 0.865...",0.787247,0.0528929
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9618585298196949, 0.957983193277311, 0.9536...",0.958999,"[0.7682403433476395, 0.7530364372469636, 0.849...",0.783317,0.0426975
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.978502080443828, 0.9789915966386554, 0.9725...",0.979771,"[0.776824034334764, 0.8421052631578947, 0.8174...",0.777725,0.0497631
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8550624133148405, 0.8529411764705882, 0.850...",0.856255,"[0.7682403433476395, 0.7125506072874493, 0.837...",0.770988,0.0413482


In [110]:
MLA_random_breath_30

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9667128987517337, 0.9565826330532213, 0.955...",0.961025,"[0.7467811158798283, 0.7813765182186235, 0.845...",0.789442,0.0388691
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.759656652360515, 0.805668016194332, 0.81349...",0.788592,0.0442591
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9993055555555...",0.999931,"[0.7682403433476395, 0.8137651821862348, 0.861...",0.786721,0.0342656
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9715672676837726, 0.9740896358543417, 0.976...",0.977265,"[0.7682403433476395, 0.805668016194332, 0.8412...",0.774512,0.0411364
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8564493758668515, 0.8431372549019608, 0.848...",0.854795,"[0.7510729613733905, 0.7368421052631579, 0.837...",0.769404,0.036008


In [111]:
MLA_random_breath_35

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.759656652360515, 0.8097165991902834, 0.8531...",0.797108,0.0296791
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9590846047156727, 0.9593837535014006, 0.951...",0.959765,"[0.759656652360515, 0.7692307692307693, 0.8492...",0.789481,0.044858
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9701803051317615, 0.9761904761904762, 0.979...",0.975596,"[0.7381974248927039, 0.8137651821862348, 0.861...",0.787469,0.0447531
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7467811158798283, 0.7611336032388664, 0.837...",0.78604,0.0426508
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8522884882108183, 0.8480392156862745, 0.841...",0.849677,"[0.7381974248927039, 0.7651821862348178, 0.825...",0.77344,0.0449497


In [112]:
MLA_random_breath_40

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.759656652360515, 0.8461538461538461, 0.8492...",0.796814,0.0476797
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9618585298196949, 0.9649859943977591, 0.956...",0.961653,"[0.759656652360515, 0.757085020242915, 0.84920...",0.789361,0.0463818
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7381974248927039, 0.8137651821862348, 0.849...",0.787136,0.0447451
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9729542302357836, 0.9761904761904762, 0.975...",0.976358,"[0.759656652360515, 0.7854251012145749, 0.8531...",0.78013,0.0525766
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8502080443828016, 0.8438375350140056, 0.839...",0.845548,"[0.7296137339055794, 0.6842105263157895, 0.809...",0.752918,0.0465382


In [113]:
MLA_random_breath_45

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9639389736477115, 0.9684873949579832, 0.959...",0.96382,"[0.7682403433476395, 0.7530364372469636, 0.825...",0.780031,0.04077
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.703862660944206, 0.805668016194332, 0.80952...",0.779191,0.0422594
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7167381974248928, 0.805668016194332, 0.8214...",0.774119,0.0487022
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9687933425797504, 0.9677871148459384, 0.969...",0.971173,"[0.7253218884120172, 0.7732793522267206, 0.837...",0.774097,0.0494285
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8474341192787794, 0.8438375350140056, 0.842...",0.845281,"[0.7725321888412017, 0.7489878542510121, 0.809...",0.753124,0.0440952


In [114]:
MLA_random_breath_50

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9660194174757282, 0.9712885154061625, 0.977...",0.974059,"[0.7467811158798283, 0.854251012145749, 0.8333...",0.79953,0.0531417
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9992997198879552, 1.0, 1.0, 0.99931553...",0.99979,"[0.721030042918455, 0.8623481781376519, 0.8095...",0.790267,0.0481682
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9667128987517337, 0.9684873949579832, 0.959...",0.965709,"[0.7725321888412017, 0.7651821862348178, 0.837...",0.784242,0.0361526
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7296137339055794, 0.8218623481781376, 0.805...",0.780773,0.0437267
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8585298196948682, 0.8543417366946778, 0.851...",0.852098,"[0.7510729613733905, 0.728744939271255, 0.7976...",0.74734,0.0358329
